In [30]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe

In [31]:
#対象被験者の特徴ベクトルをMATLABから入手
feat_l_tr = np.loadtxt("feat_left_A01T.txt",delimiter=",")
feat_r_tr = np.loadtxt("feat_right_A01T.txt",delimiter=",")
feat_Test_C1 = np.loadtxt("feat_test_left_A01T.txt",delimiter=",")
feat_Test_C2 = np.loadtxt("feat_test_right_A01T.txt",delimiter=",")

In [32]:
#非対象被験者の特徴ベクトルをMATLABから入手
feat_C1_K1 = np.loadtxt("feat_C1_K1.txt",delimiter=",")
feat_C2_K1 = np.loadtxt("feat_C2_K1.txt",delimiter=",")

In [33]:
def Dic(feat_l_tr,feat_r_tr): #入力の配列は2次元配列である必要がある
    feat_tr = np.concatenate([feat_l_tr, feat_r_tr],0)
    return feat_tr

In [34]:
#特徴ベクトルを2次元に変換
#特徴ベクトル g は２M×Tになっている必要がある
#Tは被験者の試行回数である
feat_tr = Dic(feat_l_tr,feat_r_tr)
#非対象被験者の特徴ベクトルも同様の処理を行う
feat_tr_K1 = Dic(feat_C1_K1,feat_C2_K1)
#それぞれの特徴ベクトルをつなげる
feat_tr_dic = Dic(feat_tr,feat_tr_K1).T
#feat_tr_dicはすべての特徴ベクトルをつなげたもの
#まず、分類に用いるテストデータの特徴ベクトルを2次元に変換
feat_l_te = feat_Test_C1;
feat_r_te = feat_Test_C2;
feat_l_te = feat_l_te.T
feat_r_te = feat_r_te.T
print(feat_l_te.shape,feat_tr_dic.shape,feat_tr.shape,feat_tr_K1.shape)
matrix_index = np.array([feat_l_tr.shape[0],feat_r_tr.shape[0],feat_C1_K1.shape[0],feat_C2_K1.shape[0]])

(4, 30) (4, 140) (60, 4) (80, 4)


In [35]:
def SRC(feat_l_tr,fea_r_tr,feat_tr,feat_tr_K1,feat_tr_dic,feat_C1_K1,feat_C2_K1,feat_l_te,matrix_index): #rightの時は(feat_r_tr,feat_tr,feat_r_te)
    import cvxpy as cv
    SRC_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    SRC_l_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    SRC_r_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    answer = np.zeros(feat_l_te.shape[1])
    for i in range(feat_l_te.shape[1]):
        x = cv.Variable((feat_tr_dic.shape[1],1)) #目的関数の次元
        objective = cv.Minimize(cv.norm(x,1))
        constraints = [feat_l_te[:,i,np.newaxis] == feat_tr_dic@x]
        prob = cv.Problem(objective, constraints)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付ける
        SRC_Cla[:,i] = np.squeeze(x.value)
        #パラメータxを各クラスのパラメータに分ける
        l = 0
        j = 0
        while j < matrix_index.shape[0]-1:
            SRC_l_Cla[l:matrix_index[j],i] = SRC_Cla[l:matrix_index[j],i]     
            l = l + matrix_index[j]
            j = j + 1
            SRC_r_Cla[l:l+matrix_index[j],i] = SRC_Cla[l:l+matrix_index[j],i]
            l = l + matrix_index[j]
            j = j + 1
        #分類では
        #配列xの内、右のクラスのみに関する配列を使用して作成したy(右)、左のクラスのみに関する配列を使用して作成したy(左)の
        #残差が小さい方のクラスを割り当てる
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    #sparce性(第一小数以上の値を持たないものは０にする)を重要視する場合入れる
    #SRC_l_Cla = np.asarray(SRC_l_Cla*10, dtype = int)
    #SRC_l_Cla = SRC_l_Cla*0.1
    #SRC_r_Cla = np.asarray(SRC_r_Cla*10, dtype = int)
    #SRC_r_Cla = SRC_r_Cla*0.1
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_tr_dic@SRC_l_Cla[:,i]
    for i in range(feat_r_te.shape[1]):
        dout_feat_r_te[:,i] = feat_tr_dic@SRC_r_Cla[:,i]
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    return answer

In [36]:
answer = SRC(feat_l_tr,feat_r_tr,feat_tr,feat_tr_K1,feat_tr_dic,feat_C1_K1,feat_C2_K1,feat_l_te,matrix_index)
x = 0
for i in range(answer.shape[0]):
    if answer[i] == 1:
        x = x + 1
print((x/answer.shape[0])*100)

33.33333333333333


In [37]:
answer = SRC(feat_l_tr,feat_r_tr,feat_tr,feat_tr_K1,feat_tr_dic,feat_C1_K1,feat_C2_K1,feat_r_te,matrix_index)
x = 0
for i in range(answer.shape[0]):
    if answer[i] == 2:
        x = x + 1
print((x/answer.shape[0])*100)

53.333333333333336
